# Hydrogen algorithm for QAL Quantum Inspire hybrid computing workshop 

This notebook describes running a hybrid quantum classical algorith to calculate the electronic ground state energy of a hydrogen molecule on the Quantum Inspire platform.

Part of the cells in the notebook are to be executed. Another part are to be copied to another file.

### Setting up the algorithm Python file

We start by make a new empty Python file to hold the algorithm.

In [ ]:
from pathlib import Path

filename = str(Path.cwd() / "hydrogen_algorithm.py") 
!touch {filename}

Add an `execute` and a `finalize` function to the file with the following input arguments:

In [ ]:
from typing import Any
from quantuminspire.util.api.quantum_interface import QuantumInterface


def execute(qi: QuantumInterface) -> None:
    ...

def finalize(results: Any) -> dict[str, Any]:
    ...

Have the `execute` function add something to the results, and have `finalize` return them:

In [ ]:
from typing import Any
from quantuminspire.util.api.quantum_interface import QuantumInterface


def execute(qi: QuantumInterface) -> None:
    qi.results = {"results": 2}

def finalize(results: Any) -> dict[str, Any]:
    return results

Use the `print_output` function to execute the Python file and print its output:

In [ ]:
def print_output():
    output = !poetry run qi files run {filename}
    print(output)
    
print_output()

### Adding the hydrogen algorithm

In the next couple of cells, we will be adding the hydrogen algorithm to the `execute` function. Add these commands to the function.

Specify a molecule configuration with the distance between the atoms. Make a molecule driver from this, and use it to generate an ElectronicStructureProblem.

In [ ]:
from qiskit_nature.second_q.drivers import PySCFDriver

distance = 0.735
molecule = f"H 0.0 0.0 0.0; H 0.0 0.0 {distance}"
driver = PySCFDriver(molecule)
es_problem = driver.run()

Get the fermionic (electronic creation and annihilation) operators and store some useful attributes for later 

In [ ]:
fermionic_op = es_problem.hamiltonian.second_q_op()
n_particles = es_problem.num_particles
n_spatial_orbitals = es_problem.num_spatial_orbitals
nuclear_repulsion_energy = es_problem.nuclear_repulsion_energy

Map the fermionic (electronic) operators for the Hamiltonian onto qubit operators:

In [ ]:
from qiskit_nature.second_q.mappers import ParityMapper

mapper = ParityMapper(num_particles=(1, 1))
qubit_op = mapper.map(fermionic_op)

Make the ansatz from the initial state, using the same mapper:

In [ ]:
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

initial_state = HartreeFock(n_spatial_orbitals, n_particles, mapper)
ansatz = UCCSD(n_spatial_orbitals, n_particles, mapper, initial_state=initial_state)

Use Quantum Inspire's Qiskit backend to make the estimator:

In [ ]:
from qiskit.primitives import BackendEstimator
from quantuminspire.sdk.qiskit.backend import QuantumInspireBackend

backend = QuantumInspireBackend(qi)  # qi is passed into `execute`
estimator = BackendEstimator(backend=backend)

Make the optimizer:

In [ ]:
from qiskit_algorithms.optimizers import COBYLA

optimizer = COBYLA(maxiter=1)

Put everything together to make the VQE algorithm, and run it to get the result:

In [ ]:
from qiskit_algorithms import VQE

algo = VQE(estimator, ansatz, optimizer)
result = algo.compute_minimum_eigenvalue(qubit_op)

Replace the returned result with the distance and the total energy. The total energy is the sum of the eigenvalue calculated by the algorithm and the nuclear repulsion energy. 

In [ ]:
qi.results = {"results": [{"distance": distance, "total_energy": result.eigenvalue + nuclear_repulsion_energy}]}

Print the output to show the distance and the total energy

In [ ]:
print_output()

### Varying the distance between the atoms

Now that we can calculate the total energy for a single distance, let's vary the distance and see what happens.

Wrap the entire body of the execute function in a for loop to loop over different distances, replacing the constant `distance`.

In [ ]:
import numpy as np

def execute(qi: QuantumInterface) -> None:
    distances = np.arange(0.3, 2.5, 0.1)
    results = []
    for distance in distances:
        ...  # execute body
        result_dict = {"distance": distance, "total_energy": result.eigenvalue + nuclear_repulsion_energy}
        results.append(result_dict)
    
    qi.results = {"results": results}

Print the output to see the total energy for different distances:

In [ ]:
print_output()

For a visual representation, you can plot the total energy vs the distance:

In [ ]:
from json import loads
import matplotlib.pyplot as plt

def plot_output():
    output = !poetry run qi files run {filename}
    results = loads(output[-1].replace("'", '"'))["results"]
    distances, energies = [[result[key] for result in results] for key in ["distance", "total_energy"]]
    fig, ax = plt.subplots()
    ax.plot(distances, energies)
    ax.set_xlabel("Distance (Angstrom)")
    ax.set_ylabel("Potential energy (a.u.)")
    return fig, ax

plot_output()

You can see a decreasing function in energy, the atoms start repelling each other at low distances, but it is noisy and there is no clear minimal distance.

### Changing optimizer settings

Let's try again with more iterations for the optimizer:

In [ ]:
# optimizer = COBYLA(maxiter=10)  # replace in file

plot_output()

And then some more iterations (this may take some minutes)

In [ ]:
# optimizer = COBYLA(maxiter=100)  # replace in file

plot_output()

Other optimizers are available in `qiskit_algorithms.optimizers` (https://qiskit-community.github.io/qiskit-algorithms/apidocs/qiskit_algorithms.optimizers.html). Finding the best one depends on the problem.

In [ ]:
# from qiskit_algorithms.optimizers import NELDER_MEAD
# 
# optimizer = NELDER_MEAD(maxiter=10)  # replace in file

plot_output()

### Printing the circuit

You can print the circuit that is executed. First we add the circuit to the results, in the file, next we print it with the `print_circuit` function. 

In [ ]:
from qiskit import transpile

transpiled_circuit = transpile(result.optimal_circuit, basis_gates=['ry', 'h', 'cx', 'x', 'sdg', 'rz', 's'])
result_dict["circuit"] = repr(transpiled_circuit.draw()).replace("'", '"')
result_dict["circuit_depth"] = transpiled_circuit.depth()


In [ ]:
from json import loads

def print_circuit(index: int = 0):
    output = !poetry run qi files run {filename}
    results = loads(output[-1].replace("'", '"'))["results"]
    circuits, circuit_depths = [[result[key] for result in results] for key in ["circuit", "circuit_depth"]]
    print(circuits[index])
    print(f"circuit_depth={circuit_depths[index]}")
    
print_circuit()

### Changing the qubit encoding

We can use a different encoding, e.g. the Jordan-Wigner encoding, instead of the parity encoding.

In [ ]:
# from qiskit_nature.second_q.mappers import JordanWignerMapper
# 
# mapper = JordanWignerMapper()  # replace in file

print_circuit()

### Running different optimization problems with VQE

Instead of the hydrogen molecule, the VQE algorithm can also be used for different optimization problems. For this, we will replace the `qubit_op` (what we will measure), and the `ansatz` (how we will change the state to be measured). We will measure the operator `X + distance * Z` on one qubit. We have two parameters to optimize, which perform a rotation around the y-axis and then around the z-axis.

In [ ]:
# replace in the file
from qiskit.quantum_info import SparsePauliOp

qubit_op = SparsePauliOp(["X", "Z"], coeffs=np.array([1, distance]))  # e.g. 3-qubit bitstrings could be ["XXY", "ZYX"]
nuclear_repulsion_energy = 0

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter

params = [Parameter("t_0"), Parameter("t_1")]
ansatz = QuantumCircuit(1)  # 1 = number of qubits
ansatz.ry(params[0], qubit=0)
ansatz.rz(params[1], qubit=0)

# add to the file
result_dict["optimal_t_0"] = result.optimal_parameters[params[0]]
result_dict["optimal_t_1"] = result.optimal_parameters[params[1]]

Next we will plot the optimal parameters and the calculated "energy"

In [ ]:
from json import loads
import matplotlib.pyplot as plt
import numpy as np


def plot_outputs():
    output = !poetry run qi files run {filename}
    results = loads(output[-1].replace("'", '"'))["results"]
    keys = ["distance", "total_energy", "optimal_t_0", "optimal_t_1"]
    distances, energies, optimal_t_0s, optimal_t_1s = [[result[key] for result in results] for key in keys]
    fig, ax = plt.subplots()
    ax.plot(distances, energies, label="Total Energy (a.u.)")
    ax.plot(distances, np.mod(optimal_t_0s, 2*np.pi) - np.pi, label="t_0 (rad)")
    ax.plot(distances, np.mod(optimal_t_1s, 2*np.pi) - np.pi, label="t_1 (rad)")
    ax.set_xlabel("Distance (Angstrom)")
    ax.legend()
    return fig, ax

plot_outputs()

Does it make sense what you see as optimization results?

Can you make an optimization problem with two qubits? With three parameters? With more Pauli strings?

What else can you think of?